# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [2]:
from decimal import Decimal, getcontext
from contextlib import contextmanager
from typing import Generator


@contextmanager
def set_precission(precission: int) -> Generator[None, None, None]:
    precission = int(precission)
    
    if precission < 0:
        raise ValueError
    
    #Сохранение контекста
    precission_old = getcontext().prec
    getcontext().prec = precission

    #Возвращение управления
    try:
        yield
    
    #Восстановление контекста
    finally:
        getcontext().prec = precission_old


In [3]:
num1, num2 = Decimal(3.14), Decimal(2.71)
print(num1/num2)

with set_precission(5):
    print(num1/num2)

1.158671586715867219745115392
1.1587


## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [4]:
def generator_geometry_progression(start: float = 1, step: float = 1) -> Generator[float, None, None]:
    current_member, step = int(start), int(step)

    while True:
        yield current_member
        current_member *= step

In [5]:
generator = generator_geometry_progression(step = 2)

for i in range(5):
    print(next(generator))

1
2
4
8
16


## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [6]:
def float_range(
        stop: float, 
        start: float = 0, 
        step: float = 1
        ) -> Generator[float, None, None]:
        start, stop, step = float(start), float(stop), float(step)
        current_number = float(start)
        
        if step == 0:
                raise ValueError("Step doesn't can be 0")
        
        while (current_number < stop) == (step > 0):
                yield current_number
                current_number += step

In [7]:
for i in float_range(start = 0, stop = 5, step = 0.5):
    print(i)
for i in float_range(5):
    print(i)

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
0.0
1.0
2.0
3.0
4.0


## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [20]:
from typing import Any, Callable


def my_map(
    func: Callable, 
    *args
    ) -> Generator[Any, None, None]:
    for tuple in zip(*args):
        yield func(*tuple)
        

In [19]:
for i in my_map(str, [1, 2, 3]):
    print(i+'a')

1a
2a
3a


### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [22]:
from enum import Enum
from itertools import zip_longest


class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


In [35]:
def my_map(
    func: Callable,
    *args,
    map_cutmode: MapTypes = MapTypes.SHORTEST,
    fillvalue: Any = None
    ) -> Generator[Any, None, None]:
    if map_cutmode == MapTypes.SHORTEST:
        for tuple in zip(*args):
            yield func(*tuple)
    
    else:
        for tuple in zip_longest(*args, fillvalue=fillvalue):
            yield func(*tuple)

In [83]:
for i in my_map(lambda x, y: x + y, [1, 2, 3], [1, 2], map_cutmode = MapTypes.LONGEST, fillvalue = 1):
    print(i)
for i in my_map(str, [1, 2, 3]):
    print(i+'a')

2
4
4


## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [76]:
from typing import Iterable


def generate_circle(
        iter: Iterable
) -> Generator[Any, None, None]:
    if not isinstance(iter, Iterable):
        raise ValueError('A non-iterable object was passed')
    while True:
        for obj in iter:
            yield obj

In [80]:
generator = generate_circle('abc')
for _ in range(5):
    print(next(generator))

a
b
c
a
b


### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [81]:
import time

def circle_of_sansara(
        time_limit: float,
        pause: float
) -> None:
    time_limit = float(time_limit)
    pause = float(pause)

    if pause <= 0:
        raise ValueError('Pause must be positive')
    
    generator = generate_circle(["|", "/", "—", "\\"])
    timer = 0
    now_time = time.time()
    while True:
        print(f'Thinking <{next(generator)}>', end = '\r')
        if timer + pause >= time_limit:
            time.sleep(time_limit - timer)
            return
        else:
            time.sleep(pause)
            timer = time.time() - now_time


In [82]:
circle_of_sansara(4, 0.05)